In [1]:
import requests

In [2]:
# Buscar
user_localization = [40.4165000, -3.7025600]

In [3]:
import requests

# Define the endpoint URL for train localization
endpoint_url = 'https://dadesobertes.fgc.cat/api/records/1.0/search/'

# Set up the parameters
params = {
    'dataset': 'posicionament-dels-trens',
    'rows': 100,  # Number of records to fetch
    'q': 'lin:S2'  # Replace LINE_ID with the specific line you want to filter by
}

# Make the request
response = requests.get(endpoint_url, params=params)

# Check if the response is successful
if response.status_code == 200:
    # Parse the JSON response
    train_data = response.json()       
else:
    print("Error:", response.status_code, response.text)


In [33]:
train_data['records'][0]['fields'].keys()

dict_keys(['ocupacio_m2_tram', 'lin', 'id', 'ocupacio_m1_tram', 'ocupacio_ri_tram', 'ocupacio_m2_percent', 'geo_point_2d', 'ocupacio_m1_percent', 'ocupacio_mi_percent', 'ocupacio_ri_percent', 'ocupacio_mi_tram', 'origen', 'dir', 'properes_parades', 'desti', 'en_hora', 'estacionat_a', 'ut', 'tipus_unitat'])

In [ ]:
train_data['records'][0]['fields']

In [41]:
trains = []
for data in train_data['records']:   
    geo = data['fields']['geo_point_2d']
    trains.append({'id': data['fields']['id'], 'lat': geo[0], 'lon': geo[1]})

In [5]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# User location
user_lat = 41.3851  # Example latitude
user_lon = 2.1734   # Example longitude

# Find the nearest train
nearest_train = None
min_distance = float('inf')

for train in trains:
    distance = haversine(user_lat, user_lon, train['lat'], train['lon'])
    if distance < min_distance:
        min_distance = distance
        nearest_train = train

print(f"The nearest train is {nearest_train['id']} at a distance of {min_distance:.2f} km")


NameError: name 'trains' is not defined

In [18]:
def get_trains(rows = 100, line = 'S2'):
    # Define the endpoint URL for train localization
    endpoint_url = 'https://dadesobertes.fgc.cat/api/records/1.0/search/'

    # Set up the parameters
    params = {
        'dataset': 'posicionament-dels-trens',
        'rows': rows,  # Number of records to fetch
        'q': f'lin:{line}'  # Replace LINE_ID with the specific line you want to filter by
    }

    # Make the request
    response = requests.get(endpoint_url, params=params)

    # Check if the response is successful
    if response.status_code == 200:
        # Parse the JSON response
        train_data = response.json()       
    else:
        print("Error:", response.status_code, response.text)
    return train_data

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_my_train(user_lat, user_lon, trains_information):
    trains_info = []
    for data in trains_information['records']:   
        geo = data['fields']['geo_point_2d']
        trains_info.append({'id': data['fields']['id'], 'lat': geo[0], 'lon': geo[1]})
    
    nearest_train = None
    min_distance = float('inf')

    for train in trains_info:
        distance = haversine(user_lat, user_lon, train['lat'], train['lon'])
        if distance < min_distance:
            min_distance = distance
            nearest_train = train
            
    return nearest_train

def get_train(user_lat, user_lon):
    trains_information = get_trains()
    nearest_train = get_my_train(user_lat = user_lat, 
                            user_lon = user_lon, 
                            trains_information = trains_information)
    # Get my train
    for train in trains_information['records']:
        if train['fields']['id'] == nearest_train['id']:
            my_train = train
            break
    return my_train
    
my_train = get_train(user_lat = 41.3851, user_lon = 2.1734)

In [24]:
def get_next_stations(my_train):
    stations = my_train['fields']['properes_parades']
    stations = stations.split(';')
    return eval(stations[0])['parada']
code = get_next_stations(my_train)
get_station_name_by_code(code)

'Peu del Funicular'

In [10]:
my_train.keys()

dict_keys(['datasetid', 'recordid', 'fields', 'geometry', 'record_timestamp'])

In [11]:
import requests

# Define the endpoint URL for the FGC stations dataset
endpoint_url = 'https://dadesobertes.fgc.cat/api/records/1.0/search/'

# Function to get the full name based on station code
def get_station_name_by_code(station_code):
    # Set up the parameters
    params = {
        'dataset': 'codigo-estaciones',
        'q': station_code,
        'rows': 1  # Assuming station codes are unique, fetch one record
    }

    # Make the request
    response = requests.get(endpoint_url, params=params)

    # Check if the response is successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        if data['records']:
            fields = data['records'][0]['fields']
            return fields['nom_estacio']
        else:
            return "Station not found"
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
station_code = 'PR'
station_name = get_station_name_by_code(station_code)
print(f"The full name for station code {station_code} is: {station_name}")


The full name for station code PR is: Provença


In [13]:
import requests

# URL for the FGC API endpoint that provides station data
api_url = 'https://dadesobertes.fgc.cat/api/explore/v2.1/console/'

# Assuming there is an endpoint for getting station details (you need to explore the API to find the exact one)
response = requests.get(f'{api_url}/lines/S2/stations')

# Check if the request was successful
#if response.status_code == 200:
#    stations = response.json()
#    for station in stations:
#        print(f"Station: {station['name']}, Coordinates: ({station['lat']}, {station['lng']})")
#else:
#    print(f"Failed to retrieve data: {response.status_code}")

# Example output handling (the exact fields will depend on the API response structure)


In [13]:
s2_stations = [
    "Barcelona Plaça Catalunya",
    "Provença",
    "Gràcia",
    "Sant Gervasi",
    "Muntaner",
    "La Bonanova",
    "Les Tres Torres",
    "Sarrià",
    "Peu del Funicular",
    "Baixador de Vallvidrera",
    "Les Planes",
    "La Floresta",
    "Valldoreix",
    "Sant Cugat",
    "Volpelleres",
    "Sant Joan",
    "Bellaterra",
    "Universitat Autònoma",
    "Sant Quirze",
    "Can Feu | Gràcia",
    "Sabadell Plaça Major",
    "La Creu Alta",
    "Sabadell Nord",
    "Sabadell Parc del Nord"
]

In [16]:
import requests
import json
# Define the endpoint URL for the FGC stations dataset
endpoint_url = 'https://dadesobertes.fgc.cat/api/records/1.0/search/'

# Function to get the station details based on station name
def get_station_by_name(station_name):
    # Set up the parameters
    params = {
        'dataset': 'gtfs_stops',
        'q': f'stop_name:"{station_name}"',
        'rows': 1  # Assuming station names are unique, fetch one record
    }

    # Make the request
    response = requests.get(endpoint_url, params=params)

    # Check if the response is successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        if data['records']:
            fields = data['records'][0]['fields']
            return fields
        else:
            return "Station not found"
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
s2_stations_pos = {}

for station_name in s2_stations:
    station_details = get_station_by_name(station_name)
    print(station_details)
    lat, long = station_details['stop_coordinates']
    s2_stations_pos[station_name] = {'lat': lat, 'long': long}

with open('s2_stations_pos.json', 'w') as f:
    json.dump(s2_stations_pos, f)

Error: 429, {
  "error": "You have exceeded the requests limit for anonymous users.",
  "errorcode": 10005,
  "reset_time": "2024-05-19T00:00:00Z",
  "call_limit": 5000,
  "limit_time_unit": "day"
}


TypeError: string indices must be integers

In [39]:
import json

# Load the station positions from the JSON file
with open(r'C:\Users\34644\Desktop\UABHACK\train_infor\codis_info_lineas.json', 'r') as f:
    station_positions = json.load(f)

In [40]:
station_positions.items()

dict_items([('PR', {'linia': 1.0, 'nom_linia': 'Barcelona-Vallès', 'ordre': 2.0, 'estacio': 2.0, 'nom_estacio': 'Provença'}), ('PC', {'linia': 1.0, 'nom_linia': 'Barcelona-Vallès', 'ordre': 1.0, 'estacio': 1.0, 'nom_estacio': 'Pl. Catalunya'}), ('SG', {'linia': 1.0, 'nom_linia': 'Barcelona-Vallès', 'ordre': 4.0, 'estacio': 4.0, 'nom_estacio': 'Sant Gervasi'}), ('BN', {'linia': 1.0, 'nom_linia': 'Barcelona-Vallès', 'ordre': 6.0, 'estacio': 6.0, 'nom_estacio': 'La Bonanova'}), ('MN', {'linia': 1.0, 'nom_linia': 'Barcelona-Vallès', 'ordre': 5.0, 'estacio': 5.0, 'nom_estacio': 'Muntaner'}), ('RE', {'linia': 1.0, 'nom_linia': 'Barcelona-Vallès', 'ordre': 9.0, 'estacio': 9.0, 'nom_estacio': 'Reina Elisenda'}), ('TT', {'linia': 1.0, 'nom_linia': 'Barcelona-Vallès', 'ordre': 7.0, 'estacio': 7.0, 'nom_estacio': 'Les Tres Torres'}), ('PM', {'linia': 1.0, 'nom_linia': 'Barcelona-Vallès', 'ordre': 10.0, 'estacio': 10.0, 'nom_estacio': 'Plaça Molina'}), ('VL', {'linia': 1.0, 'nom_linia': 'Barcelona

In [47]:
idx2station = {k:v['nom_estacio'] for k, v in station_positions.items()}
station2idx = {v:k for k, v in idx2station.items()}


s2_stations = [
    "Pl. Catalunya",
    "Provença",
    "Gràcia",
    "Sant Gervasi",
    "Muntaner",
    "La Bonanova",
    "Les Tres Torres",
    "Sarrià",
    "Peu del Funicular",
    "Baixador de Vallvidrera",
    "Les Planes",
    "La Floresta",
    "Valldoreix",
    "Sant Cugat",
    "Volpelleres",
    "Sant Joan",
    "Bellaterra",
    "Universitat Autònoma",
    "Sant Quirze",
    "Can Feu | Gràcia",
    "Sabadell Pl. Major",
    "Creu Alta",
    "Sabadell Nord",
    "Sabadell Parc Nord"
]

new_list = []
for station in s2_stations:
    new_list.append(station2idx[station])


In [49]:
print(new_list)

['PC', 'PR', 'GR', 'SG', 'MN', 'BN', 'TT', 'SR', 'PF', 'VL', 'LP', 'LF', 'VD', 'SC', 'VO', 'SJ', 'BT', 'UN', 'SQ', 'CF', 'PJ', 'CT', 'NO', 'PN']


In [51]:
with open(r'C:\Users\34644\Desktop\UABHACK\train_infor\posicionament-dels-trens.json', 'r', encoding='utf-8') as file:
        trains_information = json.load(file)

In [55]:
trains_information[1]

{'id': '6c4bdab144746c0fbf55c60d40|682dc7eb01',
 'lin': 'S2',
 'geo_point_2d': {'lon': 2.100968672111765, 'lat': 41.54255719841222},
 'dir': 'A',
 'origen': 'PC',
 'desti': 'PN',
 'properes_parades': '{"parada": "PJ"};{"parada": "CT"};{"parada": "NO"};{"parada": "PN"}',
 'estacionat_a': 'CF',
 'en_hora': 'True',
 'tipus_unitat': '112',
 'ut': '1f2cc5fd0372',
 'ocupacio_mi_percent': '24',
 'ocupacio_mi_tram': '1',
 'ocupacio_ri_percent': '24',
 'ocupacio_ri_tram': '1',
 'ocupacio_m1_tram': '1',
 'ocupacio_m1_percent': '24',
 'ocupacio_m2_percent': '24',
 'ocupacio_m2_tram': '1'}

In [56]:
with open(r'C:\Users\34644\Desktop\UABHACK\app\static\gtfs_stops.json', 'r') as file:
    station_positions = json.load(file)

In [59]:
station_positions

[{'stop_id': 'BN',
  'stop_name': 'La Bonanova',
  'stop_coordinates': {'lon': 2.136433966, 'lat': 41.39781569},
  'wheelchair_boarding': 0},
 {'stop_id': 'TT',
  'stop_name': 'Les Tres Torres',
  'stop_coordinates': {'lon': 2.130811822, 'lat': 41.39780042},
  'wheelchair_boarding': 0},
 {'stop_id': 'PM',
  'stop_name': 'Pl. Molina',
  'stop_coordinates': {'lon': 2.146754722, 'lat': 41.4016938},
  'wheelchair_boarding': 0},
 {'stop_id': 'VD',
  'stop_name': 'Valldoreix',
  'stop_coordinates': {'lon': 2.06831203, 'lat': 41.45784156},
  'wheelchair_boarding': 0},
 {'stop_id': 'SC',
  'stop_name': 'Sant Cugat Centre',
  'stop_coordinates': {'lon': 2.078203288, 'lat': 41.46791038},
  'wheelchair_boarding': 0},
 {'stop_id': 'RB',
  'stop_name': 'Rubí Centre',
  'stop_coordinates': {'lon': 2.031357969, 'lat': 41.48628798},
  'wheelchair_boarding': 0},
 {'stop_id': 'SQ',
  'stop_name': 'Sant Quirze',
  'stop_coordinates': {'lon': 2.088722533, 'lat': 41.52999155},
  'wheelchair_boarding': 0},


In [63]:
new_dict = {'stations': {}}
for station in station_positions:
    cord = station['stop_coordinates']
    new_dict['stations'][station['stop_id']] = {'lat': cord['lat'], 'long': cord['lon']}

In [64]:
with open("station_s2_V2.json", "w") as outfile:
    json.dump(new_dict, outfile)